In [1]:
import pandas as pd
import numpy as np
import os

from typing import Dict, List, Tuple, Union

In [2]:
df = pd.read_excel('planilhas_teste/Vigas Principais - Ponte Rio do Braço.xlsx', header=[0, 1])
df

Danos Viga Lateral Montante      Viga Lateral Jusante  \
           Unnamed: 0_level_1                    Fi   Fp                   Fi   
0   Carbonatação do concreto                    NaN  NaN                  NaN   
1       Cobrimento Deficiente                   NaN  NaN                  NaN   
2   Contaminação por Cloretos                   NaN  NaN                  NaN   
3       Corrosão de armaduras                   NaN  NaN                  NaN   
4               Desagregação                    NaN  NaN                  NaN   
5               Desplacamento                   NaN  NaN                  NaN   
6               Eflorescência                   NaN  NaN                  NaN   
7        Falha de Concretagem                   1.0  3.0                  1.0   
8                    Fissuras                   1.0  3.0                  1.0   
9                    Flechas                    NaN  NaN                  NaN   
10        Infiltração de água                   NaN  NaN                  NaN   
11                   Manchas                    NaN  NaN                  NaN   
12      Sinais de Esmagamento                   NaN  NaN                  NaN   
13        Infiltração de água                   NaN  NaN                  NaN   

         
     Fp  
0   NaN  
1   NaN  
2   NaN  
3   NaN  
4   NaN  
5   NaN  
6   NaN  
7   3.0  
8   3.0  
9   NaN  
10  NaN  
11  NaN  
12  NaN  
13  NaN

In [3]:
def adequa_dataset(path_excel: str) -> Tuple[pd.DataFrame, List[str], str]:
    """
    Adequa um conjunto de dados Excel para o formato com colunas simples e extrai os nomes dos elementos estruturais.

    :param path_excel: Caminho para o arquivo Excel.

    :return:
        - df_ajustado: DataFrame com colunas renomeadas (ex: "Fi - Elemento 1").
        - nome_elementos: Lista dos nomes dos elementos estruturais.
        - nome_arquivo: Nome do arquivo (sem caminho e sem extensão).
    """
    df_ajustado = pd.read_excel(path_excel, header=[0, 1])
    elementos_brutos = df_ajustado.columns.get_level_values(0)
    nome_elementos: List[str] = sorted(set(e for e in elementos_brutos if e != 'Danos'))

    df_ajustado.columns = [
        f"{sub} - {main}" if main != 'Danos' else 'Danos'
        for main, sub in df_ajustado.columns
    ]

    nome_arquivo = os.path.splitext(os.path.basename(path_excel))[0]

    return df_ajustado, nome_elementos, nome_arquivo

df_ajustado, nome_elementos, nome_arquivo = adequa_dataset('planilhas_teste/Vigas Principais - Ponte Rio do Braço.xlsx')
# print(nome_elementos)
# print(nome_arquivo)
df_ajustado

,Danos,Fi - Viga Lateral Montante,Fp - Viga Lateral Montante,Fi - Viga Lateral Jusante,Fp - Viga Lateral Jusante
0,Carbonatação do concreto,NaN,NaN,NaN,NaN
1,Cobrimento Deficiente,NaN,NaN,NaN,NaN
2,Contaminação por Cloretos,NaN,NaN,NaN,NaN
3,Corrosão de armaduras,NaN,NaN,NaN,NaN
4,Desagregação,NaN,NaN,NaN,NaN
5,Desplacamento,NaN,NaN,NaN,NaN
6,Eflorescência,NaN,NaN,NaN,NaN
7,Falha de Concretagem,1.0,3.0,1.0,3.0
8,Fissuras,1.0,3.0,1.0,3.0
9,Flechas,NaN,NaN,NaN,NaN


In [4]:
def avalia_elemento(df_ajustado: pd.DataFrame) -> Dict[str, Dict[str, float]]:
    """
    Avalia os elementos estruturais com base nos dados de danos e colunas Fi/Fp.

    :param df_ajustado: DataFrame com danos e colunas Fi/Fp por elemento.

    :return: Dicionário onde cada chave é um elemento e o valor é outro dicionário com:
        - 'sum_d': Soma total dos valores d.
        - 'd_max': Valor máximo de d encontrado.
        - 'g_de' : Grau de deterioração estrutural (G_de).
    """
    resultados = {}
    colunas = [col for col in df_ajustado.columns if col != "Danos"]
    elementos = sorted(set(col.split(" - ")[1] for col in colunas))

    for elemento in elementos:
        registros = []

        for _, row in df_ajustado.iterrows():
            dano = str(row["Danos"]).strip()
            if dano.lower() in ["danos", ""] or pd.isna(dano):
                continue

            try:
                fi = float(row[f"Fi - {elemento}"])
                fp = float(row[f"Fp - {elemento}"])
            except (KeyError, ValueError, TypeError):
                fi, fp = 0, 0

            if fi <= 2.0:
                d = 0.8 * fi * fp
            elif fi >= 3.0:
                d = (12 * fi - 28) * fp
            else:
                d = 0

            registros.append(d)

        sum_d = sum(registros)
        d_max = max(registros) if registros else 0
        g_de = d_max * (1 + ((sum_d - d_max) / sum_d)) if sum_d else 0

        resultados[elemento] = {
            'sum_d': sum_d,
            'd_max': d_max,
            'g_de': g_de
        }

    return resultados

resultados_elemento = avalia_elemento(df_ajustado)
resultados_elemento

{'Viga Lateral Jusante': {'sum_d': 4.800000000000001,
  'd_max': 2.4000000000000004,
  'g_de': 3.6000000000000005},
 'Viga Lateral Montante': {'sum_d': 4.800000000000001,
  'd_max': 2.4000000000000004,
  'g_de': 3.6000000000000005}}

In [5]:
def avalia_familia(df_ajustado: pd.DataFrame, nome_arquivo: str, f_r: float = 1.0) -> Dict[str, float]:
    """
    Avalia a família de elementos estruturais com base nos resultados dos elementos. 

    :param df_ajustado: DataFrame com os dados ajustados.
    :param nome_arquivo: Nome do arquivo (sem caminho e sem extensão).
    :param f_r: Fator de redução (default é 1.0).

    :return: Dicionário com os resultados:
            - gde_max: Valor máximo de g_de encontrado.
            - g_df: Grau de deterioração da família.
            - f_r: Fator de redução.
            - f_r × g_df: Produto do fator de redução pelo grau de deterioração da família.
    """
    resultados_elemento = avalia_elemento(df_ajustado)
    gde_list = [dados['g_de'] for dados in resultados_elemento.values() if dados['g_de'] > 0]

    if not gde_list:
        return {
            nome_arquivo: {
                'gde_max': 0.0,
                'g_df': 0.0,
                'f_r': f_r,
                'f_r × g_df': 0.0
            }
        }

    gde_max = max(gde_list)
    gde_sum = sum(gde_list)

    g_df = gde_max * np.sqrt(1 + (gde_sum - gde_max) / gde_sum) if gde_sum else 0
    fr_gdf = f_r * g_df

    return {
        nome_arquivo: {
            'gde_max': gde_max,
            'g_df': float(g_df),
            'f_r': f_r,
            'f_r × g_df': float(fr_gdf)
        }
    }

resumo_familia = avalia_familia(df_ajustado, nome_arquivo, f_r=5.0)
resumo_familia

{'Vigas Principais - Ponte Rio do Braço': {'gde_max': 3.6000000000000005,
  'g_df': 4.409081537009721,
  'f_r': 5.0,
  'f_r × g_df': 22.045407685048602}}

In [6]:
def avaliar_estrutura(resultados_familias: Dict[str, Dict[str, float]]) -> Tuple[float, str]:
    """
    Calcula o grau de deterioração global da estrutura (G_d) e retorna também a classificação e
    recomendação com base no valor de G_d.

    Tabela de níveis:
        -  0–15: Baixo
        - 16–50: Médio
        - 51–80: Alto
        - 81–100: Sofrível

    :param resultados_familias: Dicionário com resultados de cada família.

    :return:
        - g_d (float): Grau de deterioração global.
        - mensagem (str): Nível e ação recomendada.
    """
    numerador = 0.0
    denominador = 0.0

    for familia, dados in resultados_familias.items():
        fr = dados.get('f_r', 0)
        gdf = dados.get('g_df', 0)

        numerador += fr * gdf
        denominador += fr

    g_d = numerador / denominador if denominador else 0.0

    # Classificação baseada na tabela
    if g_d <= 15:
        nivel = "Baixo"
        recomendacao = "Estado aceitável. Manutenção preventiva."
    elif g_d <= 50:
        nivel = "Médio"
        recomendacao = "Definir prazo/natureza para nova inspeção. Planejar intervenção em longo prazo (máximo 2 anos)."
    elif g_d <= 80:
        nivel = "Alto"
        recomendacao = "Definir prazo/natureza para inspeção especializada detalhada. Planejar intervenção em médio prazo (máximo 18 meses)."
    else:
        nivel = "Sofrível"
        recomendacao = "Definir prazo/natureza para inspeção especializada detalhada. Planejar intervenção em curto prazo."

    mensagem = f"Nível de Deterioração: {nivel} (G_d = {g_d:.2f})\nAção recomendada: {recomendacao}"

    return g_d, mensagem


resumo_familias = {
    'pilar': {'gde_max': 6.2, 'g_df': 20.1, 'f_r': 1.0, 'f_r × g_df': 20.1},
    'viga': {'gde_max': 4.0, 'g_df': 60.0, 'f_r': 2.0, 'f_r × g_df': 120.0}
}

g_d, mensagem = avaliar_estrutura(resumo_familias)

print(f"G_d: {g_d:.2f}")
print(mensagem)

G_d: 46.70
Nível de Deterioração: Médio (G_d = 46.70)
Ação recomendada: Definir prazo/natureza para nova inspeção. Planejar intervenção em longo prazo (máximo 2 anos).
